In [6]:
import os, os.path
import math
import pandas as pd
import numpy as np
import psycopg2
import json
import faker
import random

In [17]:
cities = ['New York', 'Chicago', 'Los Angeles', 'San Francisco', 'Houston', 'Phoenix', 'San Diego', 'Dallas']

def generate_accounts(num_accounts):
    account_data = {
        'accountID': [],
        'name': [],
        'balance': [],
        'isBlocked': [],
        'location': []
    }
    
    for i in range(num_accounts):
        fake = faker.Faker()
        account_data['accountID'].append(i + 1)
        account_data['name'].append(fake.name())
        # random balance between 1000.00 and 10000.00 with 2 decimal places
        account_data['balance'].append(round(random.uniform(100.00, 50000.00), 2))
        account_data['isBlocked'].append(random.choice(['yes', 'no'])) # random blocked status
        account_data['location'].append(random.choice(cities)) # random location ID between 1 and number of cities

    return pd.DataFrame(account_data)

df_accounts = generate_accounts(500)
df_cities = pd.DataFrame(cities)

In [22]:
def generate_transactions(num_transactions, accounts_df):
    transaction_data = {
        'transactionID': [],
        'fromAccountID': [],
        'toAccountID': [],
        'amount': []
    }
    
    account_ids = accounts_df['accountID'].tolist()
    
    for i in range(num_transactions):
        transaction_data['transactionID'].append(i + 1)
        transaction_data['fromAccountID'].append(random.choice(account_ids))
        
        to_account_id = random.choice(account_ids)
        while to_account_id == transaction_data['fromAccountID'][-1]:  # ensure that From and To accounts are different
            to_account_id = random.choice(account_ids)
        transaction_data['toAccountID'].append(to_account_id)
        
        transaction_data['amount'].append(round(random.uniform(1.00, 5000.00), 2))  # random transaction amount between 1 and 5000

    return pd.DataFrame(transaction_data)

df_transactions = generate_transactions(5000, df_accounts)


In [33]:
df_accounts.to_csv('accounts.csv',index=False)
df_cities.to_csv('cities.csv',index=False)
df_transactions.to_csv('transactions.csv',index=False)

In [28]:
def psqlConnect(dbname, user, pwd, host, port):
    conn = None
    try:
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=pwd,
            host=host,
            port=port
        )
        print("Successfully connected to the database.")
    except psycopg2.Error as e:
        print(f"An error occurred while connecting to the database: {e}")
    return conn

def load_age():
    try:
        cur.execute("LOAD 'age';")
    except psycopg2.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    
    try:
        cur.execute("SET search_path TO ag_catalog;")
    except psycopg2.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()
        
def execute(query):
    try:
        cur.execute(query)
        conn.commit()
        rows = cur.fetchall()
        for row in rows:
            print(row)
    except psycopg2.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()

In [36]:
#replace parameters with yours 
dbname = "demo" 
user = "s9rt1v" 
pwd = "1234" 
host = "localhost" 
port = "5430"
conn = psqlConnect(dbname, user, pwd, host, port)
cur = conn.cursor()
load_age()

Successfully connected to the database.


In [42]:
#create graph
graph = 'trans'
query =f"SELECT create_graph('{graph}');"
execute(query)

In [57]:
#create Account nodes
for i in range(df_accounts.shape[0]):
    accountID = df_accounts.iloc[i]['accountID']
    name = df_accounts.iloc[i]['name']
    balance = df_accounts.iloc[i]['balance']
    isBlocked = df_accounts.iloc[i]['isBlocked']
    location = df_accounts.iloc[i]['location']
    query = f"SELECT * FROM cypher ('trans', $$ CREATE (n:Account{{accountID:{accountID},name:\"{name}\",balance:{balance},isBlocked:\'{isBlocked}\',location:\"{location}\"}})  $$) AS (a agtype);"

    execute(query)

In [49]:
#add City nodes
for i in range(df_cities.shape[0]):
    name = cities[i]
    query = f"SELECT * FROM cypher ('trans', $$ CREATE (n:City{{name:\"{name}\"}})  $$) AS (a agtype);"
    execute(query)

In [ ]:
#create isLocatedIn edges
for i in range(df_accounts.shape[0]):
    ID = int(df_accounts.iloc[i]['accountID'])
    city = df_accounts.iloc[i]['location'])
    query = f"SELECT * FROM cypher ('trans', $$ MATCH (source:Account {{ accountID: {ID} }}), (target:City {{ name: \"{city}\" }}) CREATE (source)-[:isLocatedIn]->(target)  $$) AS (a agtype);"
    execute(query)

In [55]:
#create Transfer edges
for i in range(df_transactions.shape[0]):
    fromAccountID = int(df_transactions.iloc[i]['fromAccountID'])
    toAccountID = int(df_transactions.iloc[i]['toAccountID'])
    transactionID = int(df_transactions.iloc[i]['transactionID'])
    amount = df_transactions.iloc[i]['amount']
    query = f"SELECT * FROM cypher ('trans', $$ MATCH (source:Account {{ accountID: {fromAccountID} }}), (target:Account {{ accountID: {toAccountID} }}) CREATE (source)-[:Transfer{{transactionID:{transactionID}, amount:{amount}}}]->(target)  $$) AS (a agtype);"
    execute(query)

In [1]:
from model_translator import *

In [2]:
#replace parameters with yours 
dbname = "demo" 
user = "s9rt1v" 
pwd = "1234" 
host = "localhost" 
port = "5430"
psqlConnect(dbname, user, pwd, host, port)
load_age()

Successfully connected to the database.


In [3]:
graph_name = 'trans'
translator(graph_name)

Translation Complete!!!
